In [0]:
#acessar google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

In [0]:
comentarios = pd.read_csv("/content/drive/My Drive/desafios/data_category.csv", delimiter = ',', encoding='iso-8859-1')

In [0]:
comentarios.head()

,CATEGORY,DESCRIPTION
0,CATCALLS,"03:00 pm, time when government colleges gets o..."
1,CATCALLS,1. I went to have a consultation/meeting for a...
2,CATCALLS,2 bikers whistled at me
3,CATCALLS,2 boys followed me from kamla nagar till my ho...
4,CATCALLS,2 boys on bike whistled


In [0]:
def pre_process(texto):
  texto = texto.replace(' nan ','')
  texto = texto.replace('nan ','')
  texto = texto.replace(' nan','')
  texto = re.sub('\r','',texto)
  texto = re.sub('\n','',texto)
  texto = re.sub(',',' ',texto)
  texto = re.sub('\.',' ',texto)
  texto = re.sub('\?',' ',texto)
  texto = re.sub('\!',' ',texto)
  texto = re.sub('\;',' ',texto)
  return texto

In [0]:
comentarios['new_DESCRIPTION'] = comentarios.DESCRIPTION.apply(pre_process)

In [0]:
comentarios.head()

,CATEGORY,DESCRIPTION,new_DESCRIPTION
0,CATCALLS,03:00 pm time when government colleges gets o...,03:00 pm time when government colleges gets o...
1,CATCALLS,1 I went to have a consultation/meeting for a...,1 I went to have a consultation/meeting for a...
2,CATCALLS,2 bikers whistled at me,2 bikers whistled at me
3,CATCALLS,2 boys followed me from kamla nagar till my ho...,2 boys followed me from kamla nagar till my ho...
4,CATCALLS,2 boys on bike whistled,2 boys on bike whistled


In [0]:
texto = comentarios['new_DESCRIPTION'].values
classes = comentarios['CATEGORY'].values

In [0]:
## Definido o corpus

corpus = comentarios.new_DESCRIPTION.values
corpus

array(["03:00 pm  time when government colleges gets off and students take buses to go back to their homes  On the same time  people (or more correctly 'teenage labor') try to harass school/college girls either by commenting them on their looks or by whistling  Girls during that hour avoid taking public buses  instead  they carpool with their classmates or walk majority of their way back home   Government has taken some serious actions against this  when a group of girls complained about this incident in the nearby police station ",
       "1  I went to have a consultation/meeting for an upcoming ad shoot with a Radio Jockey(quite famous)  And he attempted to rape me  but then let me go as I was screaming on top of my voice  I learnt my lesson  never go on a one on one consultations alone  We live in a really horrible world  I was just a 20 year old who walked out of that place scared and tears running down my face  But the next day when I looked back and wanted to sue him for that  I 

In [0]:
## Vetorizando o corpus (e criando nosso X)

cvec = CountVectorizer(ngram_range=(2,2),min_df=1)
x = cvec.fit_transform(corpus)
x

<10106x71920 sparse matrix of type '<class 'numpy.int64'>'
	with 214316 stored elements in Compressed Sparse Row format>

In [0]:
cvec.vocabulary_

{'03 00': 25,
 '00 pm': 8,
 'pm time': 46287,
 'time when': 61965,
 'when government': 69087,
 'government colleges': 24102,
 'colleges gets': 13750,
 'gets off': 22992,
 'off and': 42087,
 'and students': 4427,
 'students take': 56428,
 'take buses': 57035,
 'buses to': 10748,
 'to go': 62430,
 'go back': 23701,
 'back to': 7337,
 'to their': 62990,
 'their homes': 59858,
 'homes on': 28383,
 'on the': 42823,
 'the same': 59380,
 'same time': 50167,
 'time people': 61914,
 'people or': 45299,
 'or more': 43450,
 'more correctly': 38086,
 'correctly teenage': 14891,
 'teenage labor': 57598,
 'labor try': 33190,
 'try to': 64252,
 'to harass': 62462,
 'harass school': 25543,
 'school college': 50659,
 'college girls': 13658,
 'girls either': 23422,
 'either by': 18122,
 'by commenting': 11247,
 'commenting them': 14099,
 'them on': 60076,
 'on their': 42824,
 'their looks': 59878,
 'looks or': 34835,
 'or by': 43376,
 'by whistling': 11542,
 'whistling girls': 69794,
 'girls during': 23

In [0]:
## Definindo Y

y = np.array(comentarios.CATEGORY.values)
y

array(['CATCALLS', 'CATCALLS', 'CATCALLS', ..., 'TOUCHING', 'TOUCHING',
       'TOUCHING'], dtype=object)

In [0]:
vectorizer = CountVectorizer(analyzer="word")
freq_text = vectorizer.fit_transform(texto)
modelo = MultinomialNB()
modelo.fit(freq_text,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
testes = ['i was sexually molested in my house.',
         'i got on a packed bus and taking advantage of the situation started pushing his elbow towards me intentionally',
         'a guy flashed at me near the park',
         'while standing in queue it happened.',
         'This incident took place on 21st March 2013 around 6pm.']

In [0]:
freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['TOUCHING', 'TOUCHING', 'CATCALLS', 'TOUCHING', 'COMMENTING'],
      dtype='<U23')

In [0]:
resultados = cross_val_predict(modelo, freq_text, classes, cv=10)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [0]:
metrics.accuracy_score(classes,resultados)


0.4618048683950129

In [0]:
sentimento=['COMMENTING','CATCALLS','TOUCHING', 'OGLING', 'OTHERS', 'SEXUAL INVITES', 'STALKING', 'RAPE', 'CHAIN SNATCHING', 'TAKING PICTURES', 'INDECENT EXPOSURE', 'POOR', 'NORTH EAST INDIA REPORT', 'ONLINE HARASSMENT']
print (metrics.classification_report(classes,resultados,sentimento))

                         precision    recall  f1-score   support

             COMMENTING       0.44      0.53      0.48      2188
               CATCALLS       0.42      0.59      0.49      2148
               TOUCHING       0.48      0.82      0.61      1855
                 OGLING       0.51      0.20      0.28       971
                 OTHERS       0.47      0.11      0.18       580
         SEXUAL INVITES       0.47      0.26      0.33       502
               STALKING       0.28      0.03      0.05       424
                   RAPE       0.61      0.30      0.40       381
        CHAIN SNATCHING       0.75      0.44      0.56       308
        TAKING PICTURES       0.67      0.12      0.21       305
      INDECENT EXPOSURE       0.93      0.04      0.08       304
                   POOR       0.44      0.23      0.30       131
NORTH EAST INDIA REPORT       0.00      0.00      0.00         8
      ONLINE HARASSMENT       0.00      0.00      0.00         1

               accuracy

In [0]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))


Predito                  CATCALLS  CHAIN SNATCHING  ...  TOUCHING    All
Real                                                ...                 
CATCALLS                     1261                7  ...       278   2148
CHAIN SNATCHING                68              136  ...        55    308
COMMENTING                    615                8  ...       278   2188
INDECENT EXPOSURE              54                0  ...       159    304
NORTH EAST INDIA REPORT         3                0  ...         2      8
OGLING                        328                2  ...       207    971
ONLINE HARASSMENT               1                0  ...         0      1
OTHERS                        128               17  ...       158    580
POOR                           57                6  ...         6    131
RAPE                           56                1  ...       153    381
SEXUAL INVITES                 74                0  ...       163    502
STALKING                      138                0 

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [0]:

from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()

vectorizer = CountVectorizer(analyzer="word")
freq_text_l = vectorizer.fit_transform(texto)
freq_text_l.todense()
dtree.fit(freq_text_l, y)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
freq_testes_l = vectorizer.transform(testes)
dtree.predict(freq_testes_l)

array(['RAPE', 'TOUCHING', 'INDECENT EXPOSURE', 'COMMENTING', 'CATCALLS'],
      dtype=object)

In [0]:
resultados_tree = cross_val_predict(dtree, freq_text, classes, cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [0]:
metrics.accuracy_score(classes,resultados_tree)


0.45923213932317436

In [0]:
from textblob.classifiers import NaiveBayesClassifier


In [0]:
resultados = cross_val_predict(modelo, freq_text, classes, cv=10)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [0]:
metrics.accuracy_score(classes,resultados)

0.4626954284583416

In [0]:
from sklearn.ensemble import RandomForestClassifier
rcf = RandomForestClassifier(n_estimators=100)

In [0]:
rcf.fit(freq_text_l, y)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
pred = rcf.predict(X_test)

In [0]:
freq_testes_l = vectorizer.transform(testes)
rcf.predict(freq_testes_l)

array(['RAPE', 'TOUCHING', 'INDECENT EXPOSURE', 'COMMENTING',
       'COMMENTING'], dtype=object)

In [0]:
resultados_rf = cross_val_predict(rcf, freq_text, classes, cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [0]:
metrics.accuracy_score(classes,resultados_rf)

0.5158321789036217

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
modelo = Pipeline([('vect', TfidfVectorizer()), ('clf', SVC(kernel='linear', probability = True))])

In [0]:
encoder = LabelEncoder()

In [0]:

texto = pd.read_csv("/content/drive/My Drive/desafios/data_category.csv", delimiter = ',', encoding='utf-8' )
texto['DESCRIPTION'].dropna(inplace=True)
texto['CATEGORY'].dropna(inplace=True)

In [0]:
X = texto['DESCRIPTION']
X

0        03:00 pm, time when government colleges gets o...
1        1. I went to have a consultation/meeting for a...
2                                  2 bikers whistled at me
3        2 boys followed me from kamla nagar till my ho...
4                                  2 boys on bike whistled
5        2 boys started commenting about me loudly on t...
6        2 men were commenting and taking pictures of m...
7           2 men were singing obscene songs looking at me
8        2-3 boys started harrassing my friednas we bot...
9                2-3 boys were whistling at the bus stand.
10       2-3 men cat-called me and whistled while i pas...
11        2-3 men touched me inappropriately on the street
12       3 guys misbehave with me yesterday,till then i...
13       3 men were commenting and whistling at me abou...
14              3 men were commenting on me and my friend.
15       3 of my friends were coming to my home. On the...
16       3-4 boys shout at me and my friend on the trai.

In [0]:
Y = texto['CATEGORY']
Y

0        CATCALLS
1        CATCALLS
2        CATCALLS
3        CATCALLS
4        CATCALLS
5        CATCALLS
6        CATCALLS
7        CATCALLS
8        CATCALLS
9        CATCALLS
10       CATCALLS
11       CATCALLS
12       CATCALLS
13       CATCALLS
14       CATCALLS
15       CATCALLS
16       CATCALLS
17       CATCALLS
18       CATCALLS
19       CATCALLS
20       CATCALLS
21       CATCALLS
22       CATCALLS
23       CATCALLS
24       CATCALLS
25       CATCALLS
26       CATCALLS
27       CATCALLS
28       CATCALLS
29       CATCALLS
           ...   
10076    TOUCHING
10077    TOUCHING
10078    TOUCHING
10079    TOUCHING
10080    TOUCHING
10081    TOUCHING
10082    TOUCHING
10083    TOUCHING
10084    TOUCHING
10085    TOUCHING
10086    TOUCHING
10087    TOUCHING
10088    TOUCHING
10089    TOUCHING
10090    TOUCHING
10091    TOUCHING
10092    TOUCHING
10093    TOUCHING
10094    TOUCHING
10095    TOUCHING
10096    TOUCHING
10097    TOUCHING
10098    TOUCHING
10099    TOUCHING
10100    T

In [0]:
target = encoder.fit_transform(Y)


In [0]:
encoder.classes_.item(1)

'CHAIN SNATCHING'

In [0]:
target

array([ 0,  0,  0, ..., 13, 13, 13])

In [0]:
print(encoder.classes_[0])

CATCALLS


In [0]:
modelo.fit(X,target)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
            

In [0]:
modelo.predict(['a boy tried taking my breasts picture'])


array([12])

In [0]:
print(encoder.classes_[12])

TAKING PICTURES
